In [15]:
!pip install -q wandb python-dotenv pandas scikit-learn torch torchvision Pillow zipfile36 matplotlib


import os
from dotenv import load_dotenv
import zipfile
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torch
from torchvision.io import read_image

load_dotenv()
wandb = os.getenv('wanddb')

!wandb login --verify {wandb}




zsh:1: unmatched '


In [21]:
# Function to extract the dataset
def extract_dataset(zip_path,log=False):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall()
        if log: 
            print("Dataset extracted successfully!")

# Download and extract the dataset (assuming the zip file is named 'Cropped_final.zip')
extract_dataset('Cropped_final.zip')

# Define the class label mapping
label_mapping = {
    'amur_leopard': 0,
    'amur_tiger': 1,
    'birds': 2,
    'black_bear': 3,
    'brown_bear': 4,
    'dog': 5,
    'roe_deer': 6,
    'sika_deer': 7,
    'wild_boar': 8,
    'people': 9
}

# Walk through the extracted dataset folder to collect image paths and labels
image_paths = []
labels = []
for root, _, files in os.walk('dataset/Cropped_final'):
    for file in files:
        if file.lower().endswith(('.png')):
            path = os.path.join(root, file)
            image_paths.append(path)
            (lambda label_str: labels.append(label_mapping[label_str]) if label_str in label_mapping else None)(os.path.basename(root).replace('_', ' '))

# Create a DataFrame with the image paths and numerical labels
df = pd.DataFrame({'image_path': image_paths, 'label': labels})
print(f"DataFrame created with {len(df)} entries.")

# Perform a stratified random split (80% training, 20% validation)
# train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)

# Custom Dataset class for wildlife data
class WildlifeDataset(Dataset):
    def __init__(self,img_labels=label_mapping, img_dir='', transform=None, target_transform=None):
        self.img_labels = label_mapping
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label



DataFrame created with 0 entries.


In [ ]:
df

,image_path,label
0,dataset/Cropped_final/black_bear/0275_blb_d_cr...,black_bear
1,dataset/Cropped_final/black_bear/0571_blb_d_mo...,black_bear
2,dataset/Cropped_final/black_bear/0493_blb_n_cr...,black_bear
3,dataset/Cropped_final/black_bear/0429_blb_n_cr...,black_bear
4,dataset/Cropped_final/black_bear/0634_blb_n_cr...,black_bear
...,...,...
23331,dataset/__MACOSX/Cropped_final/people/._0264_p...,people
23332,dataset/__MACOSX/Cropped_final/people/._0583_p...,people
23333,dataset/__MACOSX/Cropped_final/people/._0667_p...,people
23334,dataset/__MACOSX/Cropped_final/people/._0524_p...,people
